# Higher-order functions

In this notebook, we will see that higher-order functions (HOFs) are essential modularity devices, and we will introduce the most common higher-order functions that operate over many different data structures.

In [19]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import _root_.org.scalatest._

import $ivy.$                               

import _root_.org.scalatest._

### References

[__Programming in Scala, 
A comprehensive step-by-step guide__](https://www.artima.com/shop/programming_in_scala_3ed) Third Edition.
by Martin Odersky, Lex Spoon, and Bill Venners. 

- Chapter 6. Functional objects
- Chapter 8. Functions and closures
- Chapter 16. Working with Lists
- Chapter 23. For Expressions Revisited

[__Functional programming in Scala__](https://www.manning.com/books/functional-programming-in-scala), by Paul Chiusano and Runar Bjarnason.

- Chapter 3. Functional data structures

[__Functional programming simplified__](https://alvinalexander.com/downloads/fpsimplified-free-preview.pdf), by Alvin Alexander.

- Chapter 22. Functions Are Variables, Too
- Chapter 23. Using Methods As If They Were Functions
- Chapter 24. How to Write Functions That Take Functions as Input Parameters
- Chapter 25. How to Write a ‘map’ Function
- Chapter 27. Functions Can Have Multiple Parameter Groups
- Chapter 28. Partially-Applied Functions (and Currying)

## `FoldRight`: divide and conquer

HOFs shine when the time comes to break monoliths. For instance, let's consider the following two functions:

In [2]:
def sum(list: List[Int]): Int = 
    ???

defined function sum

In [1]:
def multiply(list: List[Int]): Int = 
    ???

defined function multiply

These functions clearly share a common logic: their only differences are the value which is returned when the list is empty, and the function used to combine numbers (`+` and `*`, respectively). We can abstract away these differences and arrive to a more generic function which encodes that common logic:

which allows us to re-define in a modular way the `sum` and `multiply` functions:

In [ ]:
def sum(list: List[Int]): Int = 
    ???

In [3]:
def multiply(list: List[Int]): Int = 
    ???

defined function multiply

But we don't need to constrain ourselves to integers. In its generic version, the `combine` function is actually the `foldRight` higher-order function (for `List`s):

Graphically, we can explain the behaviour of `foldRight` as follows: 

![with elements](../images/foldRight.1.svg)

(all images credit to [Scala Visual Reference](https://superruzafa.github.io/visual-scala-reference/))

We can understand `foldRight` as an implementation of the divide-and-conquer design pattern: first, divide your problems in subproblems; second, solve these sub-problems; last, compose their solutions to obtain the solution to the overall problem. If the sub-problems are simple enough they can be solved directly; otherwise, they are solved recursively. In the case of lists:
- The problem is obtaining a value of type `B` for a given list.
- The only sub-problem corresponds to the tail of that list. 
- The arguments of the `foldRight` function tell us how to obtain the solution for the empty list (the atomic problem which can not be further decomposed), and how to obtain the solution from the solution to the sub-problem.

From this perspective, the implementation of `sum` and `multiply` is exactly the same as before when we used the function `combine`, although we can explain them differently:

In [ ]:
def sum(list: List[Int]): Int = 
    ???

In [4]:
def multiply(list: List[Int]): Int = 
    ???

defined function multiply

#### Improved type-inference

Technically, you may have noticed that we have to give extra type information in the invocations to `foldRight`. In fact, the following code doesn't compile. Check it yourself!

In [ ]:
/*
def multiply(list: List[Int]): Int = 
    foldRight(list)(1, (a, b) => a * b)
*/

In order to help the Scala compiler to infer the type parameters of the `foldRight` function, we need to change its signature a little bit: 

By splitting the second parameter list, we allow the Scala compiler to infer the type of `B`, before it analyses the type of the `cons` argument. Now this works:

#### HOFs in the Scala API

The `foldRight` HOF, as all the HOFs that we will explain in this notebook, are actually part of the standard Collections library of Scala. Typically, they are implemented as member methods of the corresponding collection class. For lists, the [Scala API](https://www.scala-lang.org/api/2.13.1/scala/collection/immutable/List.html) offers something like this:

In [2]:
object Standard{
    class List[A]{
        def foldRight[B](b: B)(f: (B, A) => B): B = ???
    }
}

defined object Standard

So, invocations of the standard `foldRight` do not receive the list as argument, but are common method invocations on some `List` instance:

Normally, we will use standard HOFs instead of our home-made versions, once they are explained. 

## The Hall of Fame of HOFs

Besides `compose`, `andThen` and `foldRight`, there are other famous higher-order functions which work great as modularity devices: 

In [ ]:
def foldLeft[A, B](list: List[A])(b: B)(f: (B, A) => B): B = ???
def filter[A](list: List[A])(f: A => Boolean): List[A] = ???
def map[A, B](list: List[A])(f: A => B): List[B] = ???
def flatMap[A, B](list: List[A])(f: A => List[B]): List[B] = ???

But most of these functions do not only work for `List`, they work also for `Option`, `Either[A, ?]`, and many other data structures. For instance:

In [ ]:
def fold[A, B](opt: Option[A])(none: B)(some: A => B): B = ???
def filter[A](opt: Option[A])(f: A => Boolean): Option[A] = ???
def map[A, B](opt: Option[A])(f: A => B): Option[B] = ???  
def flatMap[A, B](opt: Option[A])(f: A => Option[B] ): Option[B] = ???

In [ ]:
def fold[A, B, C](opt: Either[A, B])(left: A => C, right: B => C): C = ???
def filter[A, B](opt: Either[A, B])(f: B => Boolean): Either[A, B] = ???
def map[A, B, C](opt: Either[A, B])(f: B => C): Either[A, C] = ???  
def flatMap[A, B, C](opt: Either[A, B])(f: B => Either[A, C] ): Either[A, C] = ???

The `fold` function is the catamorphism for `Option` and `Either[A, ?]`, in the same way than `foldRight` is the catamorphism for lists (we will see catamorphisms later on). 

## FoldLeft: a better loop

The `foldLeft` HOF is the functional way to implement common imperative algorithms. In the following implementation, its signature declares the initial value of a mutable variable, and the update function executed in every step of the loop:

In [14]:
def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
    ???

defined function foldLeft

Similarly, the `foldLeft` function is typically used where a tail-recursive function is needed. Its recursive implementation is naturally tail-recursive:

In [6]:
//@annotation.tailrec
def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
    ???

defined function foldLeft

Using `foldLeft` we can implement functions at a higher-level of abstraction, i.e. using constructs which are nearer to the problem-domain, without caring about mutable variables, tail-safe recursion, etc. Its use also leads to very concise (and readable!) implementations. For instance, these are (almost) one-liner implementations of some of the functions of the last notebook on recursive functions:

In [7]:
def length[A](list: List[A]): Int = 
    ???

defined function length

In [8]:
def sum(list: List[Int]): Int = 
    ???

defined function sum

In [9]:
def reverse[A](list: List[A]): List[A] = 
    ???

defined function reverse

But we may also use the `foldRight` function to implement them:

In [ ]:
def length[A](list: List[A]): Int = 
    ???

In [ ]:
def sum(list: List[Int]): Int = 
    ???

In [10]:
def reverse[A](list: List[A]): List[A] = 
    ???

defined function reverse

Functionally, both versions are equivalent, but the ones using `foldLeft` are more efficient. In particular, the `foldRight` version of `reverse` has cuadratic complexity. 

#### Implementing `foldRight` with `foldLeft`

The implementation that we gave for `foldRight` was not tail-recursive, so this will blow up the stack:

In [ ]:
foldRight[Int, Int](List.fill(100000)(1))(0)((_,_)=> 0)

In order to obtain a better implementation, we may first reverse the list and use `foldLeft` as follows: 

In [11]:
def foldRightTR[A, B](list: List[A])(nil: B)(cons: (A, B) => B): B = 
    ???

defined function foldRightTR

This implementation works right with big lists:

In [12]:
// foldRightTR[Int, Int](List.fill(100000)(1))(0)((_,_)=> 0)

#### Implementing `foldLeft` with `foldRight`

This is also possible, though challenging. We start from the tail-recursive implementation of `foldLeft`. First, we rearrange the arguments so that it now returns a function instead of a plain value of type `B`. 

In [15]:
def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
    list match {
        case Nil => out
        case head :: tail => 
            foldLeft(tail)(update(out, head))(update)
    }

defined function foldLeft

This was in order to better expose the recursion pattern of `foldRight`. The solution is now obtained simply by noticing that `foldLeft(tail)(update)` is the solution for the tail:

In [16]:
def foldLeftAux[A, B](list: List[A])(update: (B, A) => B): B => B =
    ???

defined function foldLeftAux

The original signature can then be implemented as follows: 

In [17]:
def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
    ???

defined function foldLeft

## The `map` HOF

The `map` function is one of the essential HOFs. Its purpose is applying a function to the elements of a data structure, in such a way that the relationships between these elements do not change. The only thing that must be modified is the _content_ of the data structure, not its _shape_. This condition is expressed in the following equations:

1. `map(list)(identity) == list` for all `list: List[A]`
2. `map(map(list)(f))(g) == map(list)(g compose f)` for all `list: List[A]`, `f: A => B`, `g: B => C`

![map](../images/map.svg)

The implementation for lists can be done as follows:

In [18]:
def map[A, B](list: List[A])(f: A => B): List[B] = 
    ???

defined function map

The `map` function is polymorphic in `A` and `B`, but we can't pass generic functions (aka polymorphic values) as parameters using `FunctionN`classes (these classes only wrap functions from specific types to specific result types). We may create polymorphic versions of `FunctionN` clases, but in order to keep things simple, we will define the test catalogue for `map` using a specific signature chosen at random (any other may serve as well):

In [21]:
class TestMap(
    map: List[Int] => (Int => Boolean) => List[Boolean]
) extends FlatSpec with Matchers{
    
    val isEven: Int => Boolean = _ % 2 == 0
    
    "mapping an empty list" should "return an empty list" in {
        map(List())(isEven) shouldBe ???
    }
    
    "mapping an non-empty list" should "only change its content" in {
        map(List(1))(isEven) shouldBe ???
        map(List(1,2))(isEven) shouldBe ???
        map(List(1,2,3))(isEven) shouldBe ???
    }
}

defined class TestMap

To run the tests we will pass the `map` function for `Ìnt` and `Boolean` types:

In [22]:
run(new TestMap(map[Int, Boolean]))

cmd20$Helper$TestMap:
mapping an empty list
- should return an empty list *** FAILED ***
  scala.NotImplementedError: an implementation is missing
  at scala.Predef$.$qmark$qmark$qmark(Predef.scala:347)
  at ammonite.$sess.cmd17$Helper.map(cmd17.sc:2)
  at ammonite.$sess.cmd21$Helper.$anonfun$res21$2(cmd21.sc:1)
  at ammonite.$sess.cmd20$Helper$TestMap.$anonfun$new$1(cmd20.sc:8)
  at org.scalatest.OutcomeOf.outcomeOf(OutcomeOf.scala:85)
  at org.scalatest.OutcomeOf.outcomeOf$(OutcomeOf.scala:83)
  at org.scalatest.OutcomeOf$.outcomeOf(OutcomeOf.scala:104)
  at org.scalatest.Transformer.apply(Transformer.scala:22)
  at org.scalatest.Transformer.apply(Transformer.scala:20)
  at org.scalatest.FlatSpecLike$$anon$5.apply(FlatSpecLike.scala:1682)
  ...
mapping an non-empty list
- should only change its content *** FAILED ***
  scala.NotImplementedError: an implementation is missing
  at scala.Predef$.$qmark$qmark$qmark(Predef.scala:347)
  at ammonite.$sess.cmd17$Helper.map(cmd17.sc:2)
  at a

The implementation of `map` is really close to the implementation of `foldRight`. Indeed, we can give a more modular implementation using this more basic HOF: 

In [23]:
/*
def foldRight[A, B](list: List[A])(nil: B)(cons: (A, B) => B): B = 
    list match {
        case Nil => nil
        case head :: tail => cons(head, foldRight(tail)(nil)(cons))
    }
*/

def mapFR[A, B](list: List[A])(f: A => B): List[B] = 
    ???

defined function mapFR

In [ ]:
run(new TestMap(mapFR))

### `map` for `Option`

The implementation of the `map` function given above works over lists, but we can also map optional values and many other data structures. The implementation for the data structure `Option` must satisfy the following test:

In [25]:
class TestMapOption(
    map: Option[Int] => (Int => Boolean) => Option[Boolean]
) extends FlatSpec with Matchers{
    
    "mapping the `None` value" should "return `None`" in {
        map(None)(_ % 2 == 0) shouldBe ???
    }
    
    "mapping a `Some` value" should "only change its content" in {
        map(Option(3))(_ % 2 == 0) shouldBe ???
        map(Option(1))(_ % 2 == 0) shouldBe ???
        map(Option(2))(_ % 2 == 0) shouldBe ???
    }
    
        
}

defined class TestMapOption

The implementation is simple:

In [26]:
def mapOpt[A, B](maybeA: Option[A])(f: A => B): Option[B] = 
    ???

defined function mapOpt

In [ ]:
run(new TestMapOption(mapOpt))

## Filtering elements

Unlike `map`, the `filter` HOF allows us to change the shape of the data structure, removing those elements that do not satisfy a given predicate. 

![filter](../images/filter.svg)

In [27]:
def filterR[A](list: List[A])(predicate: A => Boolean): List[A] = 
   ???

defined function filterR

In [28]:
class TestFilterList(
    filter: List[Int] => (Int => Boolean) => List[Int]
) extends FlatSpec with Matchers{
    
    "filter" should "work" in {
        filter(List())(_ % 2 == 0) shouldBe List()
        filter(List(1))(_ % 2 == 0) shouldBe List()
        filter(List(1,3,5))(_ % 2 == 0) shouldBe List()
        filter(List(2,4,6))(_ % 2 == 0) shouldBe List(2,4,6)
    }
}

defined class TestFilterList

In [ ]:
run(new TestFilterList(filter))

Using `foldRight` we can get a more modular implementation: 

In [29]:
def filterFR[A](list: List[A])(predicate: A => Boolean): List[A] = 
    ???

defined function filterFR

In [ ]:
run(new TestFilterList(filterFR))

### Filtering optional values

We only have a possible value, so implementing filtering is easy in this case:

In [30]:
class TestFilterOption(
    filter: Option[Int] => (Int => Boolean) => Option[Int]
) extends FlatSpec with Matchers{
    
    "filter" should "work" in {
        filter(None)(_ % 2 == 0) shouldBe ???
        filter(Some(1))(_ % 2 == 0) shouldBe ???
        filter(Some(2))(_ % 2 == 0) shouldBe ???
    }
}

defined class TestFilterOption

In [31]:
def filter[A](maybeA: Option[A])(predicate: A => Boolean): Option[A] = 
    ???

defined function filter

In [ ]:
run(new TestFilterOption(filter))

## FlatMapping data structures

Let's consider a paragraph represented as a list of sentences, where each sentence is in turn modelled as a string made of _words_ separated by blank spaces. We want to obtain a list of all the words in each sentence. 

In [32]:
class TestWords(
    words: List[String] => List[String]
) extends FlatSpec with Matchers{
            
    val paragraph1 = List(
        "En un lugar",
        "de la Mancha", 
        "de cuyo nombre no",
        "quiero acordarme")
    
    "words" should "work" in {
        words(paragraph1) shouldBe 
            ???
    }
}

defined class TestWords

We may try to map the paragraph with a function that `split`s  each sentence into the words they are made of:

In [33]:
val paragraph: List[String] = List("en un lugar", "de la Mancha")



paragraph: List[String] = List("en un lugar", "de la Mancha")

but then we obtain a _list of lists_ of strings, not a list of plain strings. The solution is not far though: we have just to concatenate all the lists and we obtain what we need. The function `flatten` performs this concatenation:

In [34]:
def flatten[A](listOflists: List[List[A]]): List[A] = 
    ???

defined function flatten

The function `flatten` is actually provided by the Scala API. So, the implementation of the `words` function is as follows:

In [35]:
def words(paragraph: List[String]): List[String] = 
    ???

defined function words

In [ ]:
run(new TestWords(words))

This combination of the HOF `map` and the function `flatten` is so common, that it has been given a proper name: `flatMap`. 

In [36]:
def flatMap[A](list: List[A])(f: A => List[A]): List[A] = 
    ???

defined function flatMap

![filter](../images/flatMap.svg)

Using `flatMap` the word function is implemented even more easily:

In [37]:
def words(paragraph: List[String]): List[String] = 
    ???

defined function words

### FlatMapping optional values

In [38]:
def flatMap[A, B](maybeA: Option[A])(cont: A => Option[B]): Option[B] = 
    ???

defined function flatMap

In [40]:
class TestFlatMapOption(
    flatMap: Option[Int] => (Int => Option[Boolean]) => Option[Boolean]
) extends FlatSpec with Matchers {
    
    def isPositiveEven(i: Int): Option[Boolean] = 
        if (i>=0) Some(i % 2 == 0)
        else None
    
    "flatMap" should "work" in {
        flatMap(None)(isPositiveEven) shouldBe ???
        flatMap(Some(5))(isPositiveEven) shouldBe ???
        flatMap(Some(-5))(isPositiveEven) shouldBe ???
        flatMap(Some(0))(isPositiveEven) shouldBe ???
        flatMap(Some(4))(isPositiveEven) shouldBe ???      
    }
}

defined class TestFlatMapOption

In [ ]:
run(new TestFlatMapOption(flatMap))

## All together: a modularity problem

As before, we start from a list of sentences, but now we want to take care of the possible extra spaces between each word. Also, we want to obtain not the words themselves but their lengths. So, the specification of our problem is a function `lengths`:

In [41]:
class TestLengths(
    lengths: List[String] => List[Int]
) extends FlatSpec with Matchers{
            
    val paragraph1 = List(
        "En un  lugar",
        "de  la Mancha ", 
        "de cuyo nombre no",
        "quiero        acordarme")
    
    "lengths" should "work" in {
        lengths(paragraph1) shouldBe 
            ???
    }
}

defined class TestLengths

We may try to implement this function imperatively using mutable variables:

In [42]:
def lengthsM(paragraph: List[String]): List[Int] = {
    ???
}


defined function lengthsM

and this works, of course: 

In [ ]:
run(new TestLengths(lengthsM))

but can we do it better? Yes, we can! Using HOFs we can get a more concise, understandable and reliable version, with a great level of reuse!

In [43]:
def lengthsHOF(paragraph: List[String]): List[Int] = 
    ???

defined function lengthsHOF

In [ ]:
run(new TestLengths(lengthsHOF))

This solution contrasts with the mutable version in several respects:
- It's more **modular**, i.e. it's made of coarse-grained components: the HOFs `flatMap`, `filter` and `map`. The mutable version builds instead upon fine-grained components: `var`s, and `for` loops.
- The HOF components `flatMap`, etc., are actually very generic and domain-independent, and are typically part of standard libraries, so the level of **reuse** and **reliability** of the HOF-based solution is very high.
- The HOF-based solution is more **understandable**: it models the solution to the problem in terms of standard _transformations_ which are composed together using plain function composition. Moreover, these transformations are also at the right level of abstraction, i.e. it's natural to specify the solution to the problem in terms of flatMap, filter and map.

### For-comprehensions

The combination of `map`, `flatMap` and `filter` HOFs is so common, that Scala has a special syntax for them: for-comprehensions.

In [45]:
def lengthsFC(paragraph: List[String]): List[Int] = 
    ???

defined function lengthsFC

In [ ]:
run(new TestLengths(lengthsFC))